In [74]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd

import time

from datetime import date, datetime
from dateutil.rrule import rrule, DAILY, MONTHLY
from dateutil.relativedelta import relativedelta

import os.path
import os

import sys

import numpy as np
from io import StringIO

In [75]:
detailsList = []

endingDate = date(datetime.now().year, datetime.now().month, datetime.now().day)

coinDetails = {}
coinDetails['name'] = 'Bitcoin'
coinDetails['start'] = date(2013, 6, 1)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dashpay'
coinDetails['start'] = date(2014, 1, 15)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dogecoin'
coinDetails['start'] = date(2013, 1, 1)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ethereum'
coinDetails['start'] = date(2015, 8, 8)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Litecoin'
coinDetails['start'] = date(2013, 5, 5)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Monero'
coinDetails['start'] = date(2014, 5, 21)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ripple'
coinDetails['start'] = date(2013, 8, 5)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Stellar'
coinDetails['start'] = date(2014, 8, 5)
coinDetails['end'] = endingDate
detailsList.append(coinDetails)

In [76]:
def download_file(driver, start, end, name):
    time.sleep(3)
    url = 'https://trends.google.com/trends/explore?date={}%20{}&q={}'
    goURL = url.format(start, end, name)
    
    driver.get(goURL)

    try:
        elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'i.material-icons-extended.gray')))
    except TimeoutException:
        sys.exit("Too much time")

    driver.find_elements_by_xpath("//*[contains(text(), 'file_download')]")[0].click()
    time.sleep(2)
    a = 0

    while a == 0:
        downloadedFile = '{}\\data/multiTimeline.csv'.format(os.getcwd())

        if (os.path.isfile(downloadedFile)):
            a = 1

            if (name == 'Darkcoin'):
                name = 'Dashpay'

            try:
                os.rename(downloadedFile, savingFile)
            except:
                print('Error Renaming. Removing: {}'.format(savingFile))
                os.remove(downloadedFile)
        else:
            driver.find_element_by_css_selector('i.material-icons-extended.gray').click()
            print('Re clicking')
            time.sleep(2)

In [77]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", os.getcwd() + "\\data")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/json, application/x-download, text/csv, application/csv")

driver = webdriver.Firefox(firefox_profile=profile)

for coinDetail in detailsList:
    print("Scraping {} monthly from Google Trends".format(coinDetail['name']))
    
    old = coinDetail['start']
    coinDetail['start'] = coinDetail['start'] + relativedelta(months=1)
    
    for dt in rrule(MONTHLY, dtstart=coinDetail['start'], until=coinDetail['end']):
        
        print("{} {}".format(old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d')))
        
        savingFile = '{}\\data/{}-{}-{}.csv'.format(os.getcwd(), old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name'])
        
        if (not(os.path.isfile(savingFile))):
            try:

                if (old.year <= 2015):
                    if (coinDetail['name'] == 'Dashpay'):
                        if (old.year != 2015):
                            coinDetail['name'] = 'Darkcoin'
                        elif (old.year == 2015 and old.month <= 6):
                            coinDetail['name'] = 'Darkcoin'

                download_file(old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name'])
                
                if (coinDetail['name'] == 'Darkcoin'):
                    coinDetail['name'] = 'Dashpay'
                
                
            except:
                print("Sleeping 30 seconds")
                time.sleep(30)
                
        
        old  = dt

Scraping Bitcoin monthly from Google Trends
2013-06-01 2013-07-01
2013-07-01 2013-08-01
2013-08-01 2013-09-01
2013-09-01 2013-10-01
2013-10-01 2013-11-01
2013-11-01 2013-12-01
2013-12-01 2014-01-01
2014-01-01 2014-02-01
2014-02-01 2014-03-01
2014-03-01 2014-04-01
2014-04-01 2014-05-01
2014-05-01 2014-06-01
2014-06-01 2014-07-01
2014-07-01 2014-08-01
2014-08-01 2014-09-01
2014-09-01 2014-10-01
2014-10-01 2014-11-01
2014-11-01 2014-12-01
2014-12-01 2015-01-01
2015-01-01 2015-02-01
2015-02-01 2015-03-01
2015-03-01 2015-04-01
2015-04-01 2015-05-01
2015-05-01 2015-06-01
2015-06-01 2015-07-01
2015-07-01 2015-08-01
2015-08-01 2015-09-01
2015-09-01 2015-10-01
2015-10-01 2015-11-01
2015-11-01 2015-12-01
2015-12-01 2016-01-01
2016-01-01 2016-02-01
2016-02-01 2016-03-01
2016-03-01 2016-04-01
2016-04-01 2016-05-01
2016-05-01 2016-06-01
2016-06-01 2016-07-01
2016-07-01 2016-08-01
2016-08-01 2016-09-01
2016-09-01 2016-10-01
2016-10-01 2016-11-01
2016-11-01 2016-12-01
2016-12-01 2017-01-01
2017-01-01

In [78]:
def combined_trends(): 
    directory = os.getcwd() + "\\data"
    files = os.listdir(directory)
    files = pd.Series(files)

    trends = {}

    for coinDetail in detailsList:
        currentFiles = files[files.str.split('-').str[-1] == '{}.csv'.format(coinDetail['name'])]

        df = pd.DataFrame()

        for file in currentFiles:
            fullName = "{}\\{}".format(directory, file)
            details = open(fullName, "r").readlines()[3:]
            st = ''.join(details)

            try:
                tempDf = pd.read_csv(StringIO(st), header=None) 
                df = pd.concat([df, tempDf])
            except: 
                pass #because some file are empty and we removed initial lines
        
        df.columns = ['Date', 'Trend']

        df = df.set_index('Date')
        df = df[~df.index.duplicated(keep='first')]
        
        df.index = pd.to_datetime(df.index)
        
        df = df.sort_index(axis=1)
        
        trends[coinDetail['name']] = df
    
    
    return trends

In [79]:
trends = combined_trends()

In [81]:
for trend in trends:
    print("Coin: {} Starting: {} Ending: {}".format(trend, trends[trend].index[-1].strftime('%Y-%m-%d'), endingDate.strftime('%Y-%m-%d')))
    download_file(driver, trends[trend].index[-1].strftime('%Y-%m-%d'), endingDate.strftime('%Y-%m-%d'), trend)

Coin: Bitcoin Starting: 2018-05-09 Ending: 2018-05-14
Coin: Dashpay Starting: 2018-05-09 Ending: 2018-05-14
Coin: Dogecoin Starting: 2018-05-09 Ending: 2018-05-14
Coin: Ethereum Starting: 2018-05-08 Ending: 2018-05-14
Coin: Litecoin Starting: 2018-05-05 Ending: 2018-05-14
Coin: Monero Starting: 2018-05-09 Ending: 2018-05-14
Coin: Ripple Starting: 2018-05-05 Ending: 2018-05-14
Coin: Stellar Starting: 2018-05-05 Ending: 2018-05-14


In [83]:
def fix_missing(driver, trends):
    url = 'https://trends.google.com/trends/explore?date={}%20{}&q={}'
    
    for trend in trends:
        df = trends[trend]
        

        realDates = pd.date_range(start=df.index[0], end=df.index[-1])
        missing = realDates.symmetric_difference(df.index)

        if (missing.shape[0] > 0):
            
            if (trend == 'Dashpay'):
                trend = 'Darkcoin'
            
            print("\nMissing in {}".format(trend))
            
            missingVals = missing[0::30]

            for start in missingVals:
                
                if (trend == 'Dashpay'):
                    trend = 'Darkcoin'
                
                end = start + relativedelta(months=1)
                print("Starting: {} Ending: {}".format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')))
                
                goURL = url.format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), trend)
                
                driver.get(goURL)

                try:
                    elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'i.material-icons-extended.gray')))
                except TimeoutException:
                    sys.exit("Too much time")

                driver.find_elements_by_xpath("//*[contains(text(), 'file_download')]")[0].click()
                time.sleep(2)
                a = 0

                while a == 0:
                    downloadedFile = '{}\\data/multiTimeline.csv'.format(os.getcwd())

                    if (os.path.isfile(downloadedFile)):
                        a = 1

                        if (trend == 'Darkcoin'):
                            trend = 'Dashpay'

                        os.rename(downloadedFile, '{}\\data/{}-{}-{}.csv'.format(os.getcwd(), start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), trend))
                    else:
                        driver.find_element_by_css_selector('i.material-icons-extended.gray').click()
                        print('Re clicking')
                        time.sleep(2)

                old  = dt

In [84]:
fix_missing(driver, trends)

In [85]:
trends = combined_trends()

for trend in trends:
    trends[trend].to_csv("{}\\{}.csv".format(os.getcwd(), trend))